In [2]:


from deap import base
from deap import creator
from deap import gp
from deap import tools



pset = PrimitiveSet("MAIN", 2)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
pset.addTerminal(3)




pset.addPrimitive(operator.neg, 1)




expr = genFull(pset, min_=1, max_=3)
tree = PrimitiveTree(expr)

E:\Sofwares\Anaconda\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


NameError: name 'PrimitiveSet' is not defined

In [30]:
import random
import operator
import numpy as np
import pandas as pd
from binarytree import Node,build
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import operator

from deap import base
from deap import creator
from deap import gp
from deap import tools


pset = gp.PrimitiveSet("main", 9)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
a=4
pset.addTerminal(4)


pset.renameArguments(ARG0="thickness")
pset.renameArguments(ARG1="size")
pset.renameArguments(ARG2="shape")
pset.renameArguments(ARG3="ma")
pset.renameArguments(ARG4="se")
pset.renameArguments(ARG5="bn")
pset.renameArguments(ARG6="bc")
pset.renameArguments(ARG7="nn")
pset.renameArguments(ARG8="mitoses")


expr = gp.genFull(pset, min_=1, max_=5)
tree = gp.PrimitiveTree(expr)

creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=3, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)



toolbox.register("row", tools.initRepeat, list, toolbox.individual, n=10)
toolbox.register("population", tools.initRepeat, list, toolbox.row, n=10)



x=toolbox.population()
print(x[1][1])
print(x[9][9])

tree = gp.PrimitiveTree(expr)

function = gp.compile(str(x[9][9]), pset)


add(add(max(nn, add(add(4, bn), 4)), max(thickness, thickness)), max(add(ma, bc), max(mitoses, bn)))
max(mul(mul(shape, bn), mul(se, 4)), mul(max(size, ma), add(bc, shape)))


E:\Sofwares\Anaconda\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
E:\Sofwares\Anaconda\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [31]:
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn import preprocessing

data = pd.read_csv('Dataset_MLAI.txt',sep = ',',header = None)
#display(data)
data[6] =  pd.to_numeric(data[6], errors='coerce') 
# drop nas
data = data.dropna()

x_features = data.iloc[:,1:10]
y_features = data.iloc[:,10]
y_list = list(y_features.values.T.ravel())
x_array = x_features.values


x_train, x_test, y_train, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=42,shuffle = True)
# print(x_train)
x_train2, x_test, y_train2, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
# print(x_train)
x_train3, x_test, y_train3, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
# print(x_train)
x_train4, x_test, y_train4, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
# print(x_train)
x_train5, x_test, y_train5, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
# print(x_train)
x_train6, x_test, y_train6, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
# print(x_train)
x_train7, x_test, y_train7, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
# print(len(x_train))
x_train8, x_test, y_train8, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
# print(len(x_train))
x_train9, x_test, y_train9, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
# print(len(x_train))
x_train10, x_test, y_train10, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
print(len(x_train10))
x_train11, x_test, y_train11, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
x_train12, x_test, y_train12, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
x_train13, x_test, y_train13, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
x_train14, x_test, y_train14, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
x_train15, x_test, y_train15, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)


#print(X[0:5,:])

scaler = preprocessing.StandardScaler()
scaled_X = scaler.fit_transform(x_train)


l=[]
for i in range(30):
    function = gp.compile(str(x[1][1]), pset)
    if(function(scaled_X[i,0],scaled_X[i,1],scaled_X[i,2],scaled_X[i,3],scaled_X[i,4],scaled_X[i,5],scaled_X[i,6],scaled_X[i,7],scaled_X[i,8])<0):
        l.append(4)
    else:
        l.append(2)
print(l)


# summarize transformed data

#print(rescaledX[0:1,:])

30
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
